In [ ]:
import copy
import os

import matplotlib.pyplot as plt
import numpy as np
import scipy
from tqdm import tqdm

from complex_network.materials.dielectric import Dielectric
from complex_network.networks import network_factory, pole_calculator
from complex_network.networks.network_perturbator import NetworkPerturbator
from complex_network.networks.network_spec import NetworkSpec

In [ ]:
# Generate the random network
np.random.seed(1)
spec = NetworkSpec(
    network_type="delaunay",
    network_shape="circular",
    num_seed_nodes=0,
    external_offset=0.0,
    num_internal_nodes=15,
    num_external_nodes=5,
    network_size=100e-6,
    external_size=110e-6,
    node_S_mat_type="COE",
    node_S_mat_params={},
)
network = network_factory.generate_network(spec)
network.draw(show_indices=True)

In [ ]:
link_index = 9
network.add_segment_to_link(9, [0.4, 0.6])
network.draw(show_indices=True)

In [ ]:
# Network size based on internal nodes
min_x = np.min([node.position[0] for node in network.internal_nodes])
max_x = np.max([node.position[0] for node in network.internal_nodes])
min_y = np.min([node.position[1] for node in network.internal_nodes])
max_y = np.max([node.position[1] for node in network.internal_nodes])

width = max_x - min_x
height = max_y - min_y
print(f"Network width: {width/(1e-6):.2f}um")
print(f"Network height: {height/(1e-6):.2f}um")

# Mean link length
mean_length = np.mean([link.length for link in network.internal_links])
print(f"Mean link length: {mean_length/(1e-6):.2f}um")

In [ ]:
# Broad sweep to find some of the poles
Dlam = 2.5e-10
lam_centre = 550e-9
lam_min = lam_centre - Dlam
lam_max = lam_centre + Dlam

k0_min = 2 * np.pi / lam_max - 400j
k0_max = 2 * np.pi / lam_min + 0j
num_points = 1 * 10**3

x, y, data = pole_calculator.sweep(k0_min, k0_max, num_points, network)

In [ ]:
# Get positions of poles
# NOTE: the guesses were worked out by eye from looking at the figure
# If you change Dlam or lam_centre, this needs to be modified
pole_guesses = 1.142e7 + np.array(
    [
        1750 - 175j,
        2200 - 50j,
        3800 - 275j,
        6000 - 150j,
        6500 - 225j,
        7800 - 125j,
        8200 - 250j,
    ]
)
poles = np.array(
    [pole_calculator.find_pole(network, guess) for guess in pole_guesses]
)

In [ ]:
# Perturbation starts here
perturbator = NetworkPerturbator(network)

link_index = 34
Dn_values = np.linspace(1e-3, 1e-2, 100)
poles_dict, pole_shifts_dict = perturbator.track_pole_segment_n(
    poles, link_index, Dn_values
)

In [ ]:
# Plot the initial pole landscape
plot_data = np.flip(data, axis=1)

fig, ax = plt.subplots()
im = ax.imshow(
    -np.log(data),
    extent=(k0_min.real, k0_max.real, 0.0, -400),
    aspect="auto",
)
ax.scatter(np.real(poles), np.imag(poles), color="black", s=10)

In [ ]:
k0_max/400

In [ ]:
# Plot the initial pole landscape
plot_data = np.flip(data, axis=1)

fig, ax = plt.subplots()
im = ax.imshow(
    -np.log(data),
    extent=(k0_min.real, k0_max.real, 0.0, -400),
    aspect="auto",
)


# Plot shift direct
poles_direct = poles_dict["direct"]
for pole_list in poles_direct:
    xs = [p.real for p in pole_list]
    ys = [p.imag for p in pole_list]

    index = np.where(np.array(ys) < -400)[0]
    if len(index) == 0:
        index = -1
    else:
        index = index[0]

    ax.plot(xs[:index], ys[:index], color="black")
    ax.scatter(
        np.real(pole_list[:index]),
        np.imag(pole_list[:index]),
        color="white",
        s=10,
    )

# Plot shift formula
poles_formula = poles_dict["formula"]
for pole_list in poles_formula:
    xs = [p.real for p in pole_list]
    ys = [p.imag for p in pole_list]

    index = np.where(np.array(ys) < -400)[0]
    if len(index) == 0:
        index = -1
    else:
        index = index[0]

    ax.plot(xs[:index], ys[:index], color="tab:orange")

# Plot shift volume
poles_volume = poles_dict["volume"]
for pole_list in poles_volume:
    xs = [p.real for p in pole_list]
    ys = [p.imag for p in pole_list]

    index = np.where(np.array(ys) < -400)[0]
    if len(index) == 0:
        index = -1
    else:
        index = index[0]

    ax.plot(xs[:index], ys[:index], color="tab:red")

ax.scatter(np.real(poles), np.imag(poles), color="black", s=50)

In [ ]:
# Get background for zoomed in plot
# Broad sweep to find some of the poles
zoom_Dk0 = 20
zoom_k0_centre = 1.1427e7 + 800

zoom_k0_min = zoom_k0_centre - zoom_Dk0 - 180j
zoom_k0_max = zoom_k0_centre + zoom_Dk0 - 130j

zoom_x, zoom_y, zoom_data = pole_calculator.sweep(
    zoom_k0_min, zoom_k0_max, num_points, network
)

In [ ]:
# Zoom in on the right one
# Plot the initial pole landscape
# plot_data = np.flip(data, axis=1)

fig, ax = plt.subplots()
im = ax.imshow(
    -np.log(zoom_data),
    extent=(zoom_k0_min.real, zoom_k0_max.real, -130, -180),
    aspect="auto",
)


# Plot shift direct
poles_direct = poles_dict["direct"]
pole_list = poles_direct[-2]
xs = [p.real for p in pole_list]
ys = [p.imag for p in pole_list]

index = np.where(np.array(ys) < -400)[0]
if len(index) == 0:
    index = -1
else:
    index = index[0]

ax.plot(xs[:index], ys[:index], color="black")
ax.scatter(
    np.real(pole_list[:index]),
    np.imag(pole_list[:index]),
    color="white",
    s=10,
)

# Plot shift formula
poles_formula = poles_dict["formula"]
pole_list = poles_formula[-2]
xs = [p.real for p in pole_list]
ys = [p.imag for p in pole_list]

index = np.where(np.array(ys) < -400)[0]
if len(index) == 0:
    index = -1
else:
    index = index[0]

ax.plot(xs[:index], ys[:index], color="tab:orange")
ax.scatter(np.real(pole_list), np.imag(pole_list), color="white", s=5)

# Plot shift volume
poles_volume = poles_dict["volume"]
pole_list = poles_volume[-2]
xs = [p.real for p in pole_list]
ys = [p.imag for p in pole_list]

index = np.where(np.array(ys) < -400)[0]
if len(index) == 0:
    index = -1
else:
    index = index[0]

ax.plot(xs[:index], ys[:index], color="tab:red")

ax.scatter(np.real(poles[-2]), np.imag(poles[-2]), color="black", s=50)

In [ ]:
k = 2*np.pi / (550e-9)
l = network.get_link(34)
length = l.length
max_DN = 1e-2
diff = k*length* max_DN
print(diff)